In [101]:
import numpy as np
import matplotlib as plt

In [71]:
def sigmoid(z):
    return 1/(1+np.exp(-z))
    

In [145]:
def relu(z):
    if z > 0:
        return z
    else:
        return 0

In [ ]:
def get_cost(AL, Y):
    m = Y.shape[1]
    cost = np.multiply(np.log(AL),Y) + np.multiply(np.log(1 - AL),(1 - Y))
    cost = -1 * (1/m) * np.sum(cost)
    cost = np.squeeze(cost)  
    return cost

In [ ]:
def sigmoid_deriv(z):
    pass

In [ ]:
def relu_deriv(z):
    pass

In [166]:
class NeuralNet():
    def __init__(self, layers_dims):
        self.layers_dims = layers_dims
        self.ls = len(layers_dims)
        self.params = {}
        for l in range(1, ls):
            print(l)
            params['W' + str(l)] = np.random.randn(layers_dims[l], layers_dims[l-1])
            params['b' + str(l)] = np.zeros((layers_dims[l], 1))
        print(params)
        
        
    def forward_prop(self, X, Y):
        costs = []
        Z_list = []
        A_list = []
        layers = self.ls
        A_prev = X
        for l in range(1, layers):
            W = self.params['W' + str(l)]
            b = self.params['b' + str(l)]
            Z = np.dot(W, A_prev) + b
            Z_list.append(Z)
            
            g = None
            if l == layers - 1:
                g = sigmoid
            else:
                g == relu
            A = g(Z)
            A_list.append(A_prev)
            J = get_cost(A, Y)
            costs.append(J)
            A_prev = A
        return costs, Z_list, A_list
           
        
        
        
        
    def back_prop(self, cache):
        grads = {}
        layers = self.ls
        for l in reversed(range(1, layers)):
            Zl = cache[0][l]
            A = cache[1][l]
            dA = 
            g_back = None
            if l == layers:
                g_back = sigmoid_deriv
            else:
                g_back = relu_deriv
                
            dZ = dA * g_back(Zl)
            dW = (1/m)*np.dot(Z, A.T)
            db = (1/m)*np.sum(dZ, axis=1, keepdims=True)
        
            W = self.params['W'+str(l)]
            dA_prev = np.dot(W.T, dZ)
        
            dA = dA_prev
        
            grads['dW'+str(l)] = dW
            grads['db'+str(l)] = db
        return grads

    
    
    def fit(self, X, Y, learning_rate, epochs):
        for i in range(epochs):
            for_pass = self.forward_prop(X, Y)  #returns costs, Zs
            cache = (for_pass[1], for_pass[2])
            grads = self.back_prop(cache)
            lr = learning_rate
            for l in range(self.ls):
                self.params['W'+ str(l+1)] = self.params['W'+ str(l+1)] - lr*grads['dW'+str(l+1)]
                self.params['b'+ str(l+1)] = self.params['b'+ str(l+1)] - lr*grads['db'+str(l+1)]
        
    
        
    

In [155]:
layers_dims = [4, 3, 2]
#neural_net(layers_dims)
back_prop(5)

4
3
2
1
